In [1]:
from functools import partial
import random
import typing as tp

import torch
from torch import nn
from torch.nn import functional as F
import torchaudio as ta

from bm.models.common import (
    ConvSequence, ScaledEmbedding, SubjectLayers,
    DualPathRNN, ChannelMerger, ChannelDropout, pad_multiple
)
from bm.models import SimpleConv
from bm import play

Hostname bagration not defined in /conf/study_paths/study_paths.yaml. Using default paths.


In [12]:
simpleconv={
      "hidden":
        {"meg": 320},
      "batch_norm": True,
      "depth": 10,
      "dilation_period": 1,
      "kernel_size": [3,3,3,3,3,3,3,3,3,3],
      "skip": True, 
      "subject_layers": False,
      "subject_dim": 0,
      "complex_out": False, 
      "glu": 2, 
      "glu_context": 1, 
      "merger": False,
      "initial_linear": 270,
      "avg_pool_out" : False,
      "adaptive_pooling_size" : 1,
      "strides" : [1,1,1,1,1,1,1,1,1,1],
      "padding": [1,1,1,1,1,1,1,1,1,1,1,1],
      "gelu": True,
      "merger_pos_dim": 2048,
      "auto_padding" : False,
      "flatten_out" : True,
       "conv_dropout" : 0.2,
     "is_deformable_conv": True,
       "seq_len" : 361, 
      "flatten_out_channels" : 512}

In [13]:
# simpleconv={
#       "hidden":
#         {"meg": 320},
#       "batch_norm": True,
#       "depth": 6,
#       "dilation_period": 1,
#       "kernel_size": [11,9,7,5,5,5,3,3,3,3],
#       "skip": True, 
#       "subject_layers": False,
#       "subject_dim": 0,
#       "complex_out": False, 
#       "glu": 2, 
#       "glu_context": 1, 
#       "merger": False,
#       "initial_linear": 270,
#       "avg_pool_out" : False,
#       "adaptive_pooling_size" : 1,
#       "strides" : [2,1,1,1,1,1,1,1,1,1],
#       "padding": [0,0,0,0,0,0,0,0,0,0,0,0],
#       "gelu": True,
#       "merger_pos_dim": 2048,
#       "auto_padding" : False,
#       "flatten_out" : True,
#        "conv_dropout" : 0.2,
#        "seq_len" : 361, 
#       "flatten_out_channels" : 512}

In [14]:
in_channels = {"meg" : 207}
model_chout =1024

In [15]:
len([1,1,1,1,1,1,1,1,1,1])

10

In [16]:
model = SimpleConv(in_channels=in_channels, out_channels=model_chout,
                           n_subjects=27, **simpleconv)

In [17]:
batch =torch.rand(size=(256, 207, 362))

In [18]:
class SegmentBatch():
    def __init__(self, indx, batch):
        self.subject_index =indx
        self.meg = batch

In [19]:
segment_batch =SegmentBatch(0, batch)

In [20]:
out =model({"meg" : batch}, segment_batch)

Crop & pad:  361


In [21]:
out.shape

torch.Size([256, 512, 361])

In [27]:
from torch import nn

In [13]:
m = nn.AdaptiveAvgPool1d(1)

In [14]:
input = torch.randn(1, 64, 8)
output = m(input)

In [15]:
output.shape

torch.Size([1, 64, 1])

In [16]:
from bm.models.features import ConvWave

In [17]:
feature_model =ConvWave()

ConvWave


In [18]:
feature_model

ConvWave(
  (first_layer): Conv1d(1024, 512, kernel_size=(3,), stride=(1,), padding=(1,))
  (dropout): Dropout(p=0.2, inplace=False)
)

In [19]:
batch =torch.rand(size=(256, 1024, 361))

In [20]:
out =feature_model(batch)

In [21]:
out.shape

torch.Size([256, 512, 361])

In [2]:
from bm.models import TimesNet

In [3]:
conf_timesnet ={
      "hidden":
        {"meg": 320},
      "depth": 2,
      "subject_layers": False,
      "sequence_lenth" : 361,
      "num_kernels" : 6,
      "top_k": 3,
      "dropout_projection": 0.3,
      "d_model": 32,
      "d_ff": 32
    }

In [4]:
in_channels = {"meg" : 207}
model_chout =1024

In [5]:
model = TimesNet(in_channels=in_channels, out_channels=model_chout,
                           n_subjects=27, **conf_timesnet)

In [6]:
model

TimesNet(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(207, 32, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TemporalEmbedding(
      (hour_embed): FixedEmbedding(
        (emb): Embedding(24, 32)
      )
      (weekday_embed): FixedEmbedding(
        (emb): Embedding(7, 32)
      )
      (day_embed): FixedEmbedding(
        (emb): Embedding(32, 32)
      )
      (month_embed): FixedEmbedding(
        (emb): Embedding(13, 32)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (model): ModuleList(
    (0-1): 2 x TimesBlock(
      (conv): Sequential(
        (0): Inception_Block_V1(
          (kernels): ModuleList(
            (0): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
            (1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (2): Conv2d(32, 32, kernel_size=(5

In [12]:
batch =torch.rand(size=(256, 207, 361))

In [13]:
class SegmentBatch():
    def __init__(self, indx, batch):
        self.subject_index =indx
        self.meg = batch

In [14]:
segment_batch =SegmentBatch(0, batch)

In [15]:
out =model({"meg" : batch}, segment_batch)

torch.Size([256, 207, 361])


In [11]:
batch.shape

torch.Size([256, 207, 365])

In [16]:
F.interpolate(batch, 368).shape

torch.Size([256, 207, 368])

In [12]:
out.size()

torch.Size([256, 1024, 360])

In [13]:
import torch.nn.functional as F

In [23]:
F.pad(batch, (0, 2)).shape

torch.Size([256, 207, 362])

In [24]:
batch.shape

torch.Size([256, 207, 360])